# Hotel Support Agent Tutorial

This notebook demonstrates the Agent Catalog hotel support agent using LangChain with Couchbase vector store and Arize Phoenix evaluation.


## Setup and Imports

Import all necessary modules for the hotel support agent.


In [1]:
import base64
import getpass
import json
import logging
import os
import sys
import time
from datetime import timedelta

import agentc
import agentc_langchain
import dotenv
import requests
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Setup logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Suppress verbose logging
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("agentc_core").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)

# Set default values for travel-sample bucket configuration
DEFAULT_BUCKET = "travel-sample"
DEFAULT_SCOPE = "agentc_data"
DEFAULT_COLLECTION = "hotel_data"
DEFAULT_INDEX = "hotel_data_index"


## Environment Setup

Setup environment variables and configuration for the hotel support agent.


In [2]:
def setup_capella_ai_config():
    """Setup Capella AI configuration - requires environment variables to be set."""
    required_capella_vars = [
        "CB_USERNAME",
        "CB_PASSWORD", 
        "CAPELLA_API_ENDPOINT",
        "CAPELLA_API_EMBEDDING_MODEL",
        "CAPELLA_API_LLM_MODEL",
    ]
    missing_vars = [var for var in required_capella_vars if not os.getenv(var)]
    if missing_vars:
        raise ValueError(f"Missing required Capella AI environment variables: {missing_vars}")

    return {
        "endpoint": os.getenv("CAPELLA_API_ENDPOINT"),
        "embedding_model": os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
        "llm_model": os.getenv("CAPELLA_API_LLM_MODEL"),
    }


def test_capella_connectivity():
    """Test connectivity to Capella AI services."""
    try:
        endpoint = os.getenv("CAPELLA_API_ENDPOINT")
        if not endpoint:
            logger.warning("CAPELLA_API_ENDPOINT not configured")
            return False

        # Test embedding model (requires API key)
        if os.getenv("CB_USERNAME") and os.getenv("CB_PASSWORD"):
            api_key = base64.b64encode(
                f"{os.getenv('CB_USERNAME')}:{os.getenv('CB_PASSWORD')}".encode()
            ).decode()

            headers = {
                "Authorization": f"Basic {api_key}",
                "Content-Type": "application/json",
            }

            # Test embedding
            logger.info("Testing Capella AI connectivity...")
            embedding_data = {
                "model": os.getenv("CAPELLA_API_EMBEDDING_MODEL", "intfloat/e5-mistral-7b-instruct"),
                "input": "test connectivity",
            }

            response = requests.post(
                f"{endpoint}/embeddings", json=embedding_data, headers=headers
            )
            if response.status_code == 200:
                logger.info("✅ Capella AI embedding test successful")
                return True
            else:
                logger.warning(f"❌ Capella AI embedding test failed: {response.text}")
                return False
        else:
            logger.warning("Capella AI credentials not available")
            return False
    except Exception as e:
        logger.warning(f"❌ Capella AI connectivity test failed: {e}")
        return False


def _set_if_undefined(env_var: str, default_value: str = None):
    """Set environment variable if not already defined."""
    if not os.getenv(env_var):
        if default_value is None:
            value = getpass.getpass(f"Enter {env_var}: ")
        else:
            value = default_value
        os.environ[env_var] = value


def setup_environment():
    """Setup required environment variables with defaults for travel-sample configuration."""
    logger.info("Setting up environment variables...")

    # Set default bucket configuration
    _set_if_undefined("CB_BUCKET", DEFAULT_BUCKET)
    _set_if_undefined("CB_SCOPE", DEFAULT_SCOPE)
    _set_if_undefined("CB_COLLECTION", DEFAULT_COLLECTION)
    _set_if_undefined("CB_INDEX", DEFAULT_INDEX)

    # Required Couchbase connection variables
    _set_if_undefined("CB_CONN_STRING")
    _set_if_undefined("CB_USERNAME")
    _set_if_undefined("CB_PASSWORD")

    # Optional Capella AI configuration
    endpoint = os.getenv("CAPELLA_API_ENDPOINT")
    if endpoint:
        # Ensure endpoint has /v1 suffix for OpenAI compatibility
        if not endpoint.endswith("/v1"):
            endpoint = endpoint.rstrip("/") + "/v1"
            os.environ["CAPELLA_API_ENDPOINT"] = endpoint
            logger.info(f"Added /v1 suffix to endpoint: {endpoint}")

    # Test Capella AI connectivity
    test_capella_connectivity()


# Setup environment
setup_environment()


2025-07-28 14:40:11,568 - __main__ - INFO - Setting up environment variables...
2025-07-28 14:40:11,569 - __main__ - INFO - Added /v1 suffix to endpoint: https://6u2le4tknje7yzkg.ai.cloud.couchbase.com/v1
2025-07-28 14:40:11,570 - __main__ - INFO - Testing Capella AI connectivity...
2025-07-28 14:40:12,767 - __main__ - INFO - ✅ Capella AI embedding test successful


## CouchbaseClient Class

Define the CouchbaseClient for all database operations.


In [3]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)

            # Use WAN profile for better timeout handling with remote clusters
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=10))
            logger.info("Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"Failed to connect to Couchbase: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str):
        """Setup collection - create scope and collection if they don't exist, but don't clear scope."""
        try:
            # Ensure cluster connection
            if not self.cluster:
                self.connect()

            # For travel-sample bucket, assume it exists
            if not self.bucket:
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"Connected to bucket '{self.bucket_name}'")

            # Setup scope
            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"Scope '{scope_name}' created successfully")

            # Setup collection - clear if exists, create if doesn't
            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists:
                logger.info(f"Collection '{collection_name}' exists, clearing data...")
                # Clear existing data
                self.clear_collection_data(scope_name, collection_name)
            else:
                logger.info(f"Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"Collection '{collection_name}' created successfully")

            time.sleep(3)

            # Create primary index
            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
                logger.info("Primary index created successfully")
            except Exception as e:
                logger.warning(f"Error creating primary index: {e}")

            logger.info("Collection setup complete")
            return self.bucket.scope(scope_name).collection(collection_name)

        except Exception as e:
            raise RuntimeError(f"Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str):
        """Clear all data from a collection."""
        try:
            logger.info(f"Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")
            
            # Use N1QL to delete all documents with explicit execution
            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)
            
            # Execute the query and get the results
            rows = list(result)
            
            # Wait a moment for the deletion to propagate
            time.sleep(2)
            
            # Verify collection is empty
            count_query = f"SELECT COUNT(*) as count FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            count_result = self.cluster.query(count_query)
            count_row = list(count_result)[0]
            remaining_count = count_row['count']
            
            if remaining_count == 0:
                logger.info(f"Collection cleared successfully, {remaining_count} documents remaining")
            else:
                logger.warning(f"Collection clear incomplete, {remaining_count} documents remaining")
            
        except Exception as e:
            logger.warning(f"Error clearing collection data: {e}")
            # If N1QL fails, try to continue anyway
            pass

    def get_collection(self, scope_name: str, collection_name: str):
        """Get a collection object."""
        key = f"{scope_name}.{collection_name}"
        if key not in self._collections:
            self._collections[key] = self.bucket.scope(scope_name).collection(collection_name)
        return self._collections[key]

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index."""
        try:
            scope_index_manager = self.bucket.scope(scope_name).search_indexes()

            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"Vector search index '{index_name}' already exists")
        except Exception as e:
            raise RuntimeError(f"Error setting up vector search index: {e!s}")

    def load_hotel_data(self, scope_name, collection_name, index_name, embeddings):
        """Load hotel data into Couchbase."""
        try:
            # Import hotel data loading function
            sys.path.append(os.path.join(os.getcwd(), "data"))
            from hotel_data import load_hotel_data_to_couchbase

            # Load hotel data using the data loading script
            load_hotel_data_to_couchbase(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embeddings=embeddings,
                index_name=index_name,
            )
            logger.info("Hotel data loaded into vector store successfully")

        except Exception as e:
            raise RuntimeError(f"Error loading hotel data: {e!s}")

    def setup_vector_store(
        self, scope_name: str, collection_name: str, index_name: str, embeddings
    ):
        """Setup vector store with hotel data."""
        try:
            # Load hotel data
            self.load_hotel_data(scope_name, collection_name, index_name, embeddings)

            # Create vector store instance
            vector_store = CouchbaseVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embedding=embeddings,
                index_name=index_name,
            )

            logger.info("Vector store setup complete")
            return vector_store

        except Exception as e:
            raise RuntimeError(f"Error setting up vector store: {e!s}")


## Hotel Support Agent Setup

Setup the hotel support agent with Agent Catalog integration.


In [4]:
def setup_hotel_support_agent():
    """Setup the hotel support agent with Agent Catalog integration."""
    try:
        # Initialize Agent Catalog with single application span
        catalog = agentc.catalog.Catalog()
        application_span = catalog.Span(name="Hotel Support Agent")

        # Setup Couchbase connection and collections
        couchbase_client = CouchbaseClient(
            conn_string=os.getenv("CB_CONN_STRING"),
            username=os.getenv("CB_USERNAME"),
            password=os.getenv("CB_PASSWORD"),
            bucket_name=os.getenv("CB_BUCKET", DEFAULT_BUCKET),
        )
        couchbase_client.connect()
        couchbase_client.setup_collection(
            os.getenv("CB_SCOPE", DEFAULT_SCOPE),
            os.getenv("CB_COLLECTION", DEFAULT_COLLECTION)
        )

        # Setup vector index
        try:
            with open("agentcatalog_index.json", "r") as file:
                index_definition = json.load(file)
            logger.info("Loaded vector search index definition from agentcatalog_index.json")
        except Exception as e:
            raise ValueError(f"Error loading index definition: {e!s}")
        
        couchbase_client.setup_vector_search_index(
            index_definition, os.getenv("CB_SCOPE", DEFAULT_SCOPE)
        )

        # Setup embeddings using Capella AI
        try:
            if (
                os.getenv("CB_USERNAME")
                and os.getenv("CB_PASSWORD")
                and os.getenv("CAPELLA_API_ENDPOINT")
                and os.getenv("CAPELLA_API_EMBEDDING_MODEL")
            ):
                api_key = base64.b64encode(
                    f"{os.getenv('CB_USERNAME')}:{os.getenv('CB_PASSWORD')}".encode()
                ).decode()
                embeddings = OpenAIEmbeddings(
                    model=os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
                    api_key=api_key,
                    base_url=os.getenv("CAPELLA_API_ENDPOINT"),
                )
                logger.info("✅ Using Capella AI for embeddings")
            else:
                raise ValueError("Capella AI credentials not available")
        except Exception as e:
            logger.error(f"❌ Capella AI embeddings failed: {e}")
            raise RuntimeError("Capella AI embeddings required for this configuration")

        couchbase_client.setup_vector_store(
            os.getenv("CB_SCOPE", DEFAULT_SCOPE),
            os.getenv("CB_COLLECTION", DEFAULT_COLLECTION),
            os.getenv("CB_INDEX", DEFAULT_INDEX),
            embeddings,
        )

        # Setup LLM with Agent Catalog callback - try Capella AI first, fallback to OpenAI
        try:
            # Create API key for Capella AI
            api_key = base64.b64encode(
                f"{os.getenv('CB_USERNAME')}:{os.getenv('CB_PASSWORD')}".encode()
            ).decode()

            llm = ChatOpenAI(
                api_key=api_key,
                base_url=os.getenv("CAPELLA_API_ENDPOINT"),
                model=os.getenv("CAPELLA_API_LLM_MODEL"),
                temperature=0,
                callbacks=[agentc_langchain.chat.Callback(span=application_span)],
            )
            # Test the LLM works
            llm.invoke("Hello")
            logger.info("✅ Using Capella AI LLM")
        except Exception as e:
            logger.warning(f"⚠️ Capella AI LLM failed: {e}")
            logger.info("🔄 Falling back to OpenAI LLM...")
            _set_if_undefined("OPENAI_API_KEY")
            llm = ChatOpenAI(
                api_key=os.getenv("OPENAI_API_KEY"),
                model="gpt-4o",
                temperature=0,
                callbacks=[agentc_langchain.chat.Callback(span=application_span)],
            )
            logger.info("✅ Using OpenAI LLM as fallback")

        # Load tools and create agent
        tool_search = catalog.find("tool", name="search_vector_database")
        if not tool_search:
            raise ValueError(
                "Could not find search_vector_database tool. Make sure it's indexed with 'agentc index tools/'"
            )

        tools = [
            Tool(
                name=tool_search.meta.name,
                description=tool_search.meta.description,
                func=tool_search.func,
            ),
        ]

        hotel_prompt = catalog.find("prompt", name="hotel_search_assistant")
        if not hotel_prompt:
            raise ValueError(
                "Could not find hotel_search_assistant prompt in catalog. Make sure it's indexed with 'agentc index prompts/'"
            )

        custom_prompt = PromptTemplate(
            template=hotel_prompt.content.strip(),
            input_variables=["input", "agent_scratchpad"],
            partial_variables={
                "tools": "\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
                "tool_names": ", ".join([tool.name for tool in tools]),
            },
        )

        def handle_parsing_error(error) -> str:
            """Custom error handler for parsing errors that guides agent back to ReAct format."""
            logger.warning(f"Parsing error occurred: {error}")
            return """I need to use the correct format. Let me start over:

Thought: I need to search for hotels using the search_vector_database tool
Action: search_vector_database
Action Input: """

        agent = create_react_agent(llm, tools, custom_prompt)

        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=handle_parsing_error,
            max_iterations=8,
            max_execution_time=120,
            early_stopping_method="force",
            return_intermediate_steps=True,
        )

        return agent_executor, application_span

    except Exception as e:
        logger.exception(f"Error setting up hotel support agent: {e}")
        raise


# Setup the hotel support agent
agent_executor, application_span = setup_hotel_support_agent()


2025-07-28 14:40:14,979 - __main__ - INFO - Successfully connected to Couchbase
2025-07-28 14:40:16,092 - __main__ - INFO - Connected to bucket 'travel-sample'
2025-07-28 14:40:17,920 - __main__ - INFO - Collection 'hotel_data' exists, clearing data...
2025-07-28 14:40:17,921 - __main__ - INFO - Clearing data from travel-sample.agentc_data.hotel_data...
2025-07-28 14:40:22,477 - __main__ - INFO - Collection cleared successfully, 0 documents remaining
2025-07-28 14:40:26,476 - __main__ - INFO - Primary index created successfully
2025-07-28 14:40:26,477 - __main__ - INFO - Collection setup complete
2025-07-28 14:40:26,488 - __main__ - INFO - Loaded vector search index definition from agentcatalog_index.json
2025-07-28 14:40:27,391 - __main__ - INFO - Vector search index 'hotel_data_index' already exists
2025-07-28 14:40:27,721 - __main__ - INFO - ✅ Using Capella AI for embeddings
2025-07-28 14:40:30,544 - hotel_data - INFO - Loading hotel data from travel-sample.inventory.hotel...
2025-0

## Test Functions

Define test functions to demonstrate the hotel support agent functionality.


In [5]:
def run_hotel_query(query: str, agent_executor, application_span):
    """Run a single hotel query with error handling."""
    logger.info(f"🔍 Hotel Query: {query}")
    
    try:
        with application_span.new(f"Hotel Query: {query}") as query_span:
            query_span["query"] = query
            
            # Run the agent
            response = agent_executor.invoke({"input": query})
            result = response.get("output", "No response generated")
            
            query_span["result"] = result
            logger.info(f"🤖 AI Response: {result}")
            logger.info("✅ Query completed successfully")
            
            return result
            
    except Exception as e:
        logger.exception(f"❌ Query failed: {e}")
        return f"Error: {str(e)}"


def test_hotel_data_loading():
    """Test hotel data loading from travel-sample independently."""
    logger.info("Testing Hotel Data Loading from travel-sample")
    logger.info("=" * 50)
    
    try:
        # Import hotel data functions
        sys.path.append(os.path.join(os.getcwd(), "data"))
        from hotel_data import get_hotel_count, get_hotel_texts
        
        # Test hotel count
        count = get_hotel_count()
        logger.info(f"✅ Hotel count in travel-sample.inventory.hotel: {count}")
        
        # Test hotel text generation
        texts = get_hotel_texts()
        logger.info(f"✅ Generated {len(texts)} hotel texts for embeddings")
        
        if texts:
            logger.info(f"✅ First hotel text sample: {texts[0][:200]}...")
        
        logger.info("✅ Data loading test completed successfully")
        
    except Exception as e:
        logger.exception(f"❌ Data loading test failed: {e}")


# Import queries for consistent testing
sys.path.append(os.path.join(os.getcwd(), "data"))
from queries import get_evaluation_queries

# Test hotel data loading
test_hotel_data_loading()


2025-07-28 14:41:24,863 - __main__ - INFO - Testing Hotel Data Loading from travel-sample
2025-07-28 14:41:24,864 - __main__ - INFO - ==================================================
2025-07-28 14:41:27,089 - __main__ - INFO - ✅ Hotel count in travel-sample.inventory.hotel: 917
2025-07-28 14:41:29,131 - hotel_data - INFO - Loading hotel data from travel-sample.inventory.hotel...
2025-07-28 14:41:31,519 - hotel_data - INFO - Loaded 917 hotels from travel-sample.inventory.hotel
Processing hotels: 100%|██████████| 917/917 [00:00<00:00, 138755.97it/s]
2025-07-28 14:41:31,528 - hotel_data - INFO - Generated 917 hotel text embeddings
2025-07-28 14:41:31,530 - __main__ - INFO - ✅ Generated 917 hotel texts for embeddings
2025-07-28 14:41:31,531 - __main__ - INFO - ✅ First hotel text sample: 'La Mirande Hotel in Avignon, France. Title: Avignon. Description: 5 star hotel housed in a 700 year old converted townhouse. Address: 4 place de la Mirande,F- AVIGNON. Price: €400 and up. State: Prov...


## Test 1: Hotel Search in Giverny

Search for hotels in Giverny with free breakfast.


In [6]:
# Get evaluation queries for consistent testing
eval_queries = get_evaluation_queries()

result1 = run_hotel_query(
    eval_queries[0],  # Giverny with free breakfast
    agent_executor,
    application_span
)


2025-07-28 14:41:31,535 - __main__ - INFO - 🔍 Hotel Query: Find hotels in Giverny with free breakfast




> Entering new AgentExecutor chain...
Okay, so I need to find hotels in Giverny with free breakfast. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity. That means I should phrase my query in a way that matches what people might search for when looking for hotels.

I think starting with "Giverny hotels with free breakfast" makes sense. It's straightforward and covers both the location and the specific amenity they're looking for. I should make sure to include all the important keywords so the tool can find the most relevant results.

I wonder if there are any other ways to phrase it, but I think this is clear enough. Maybe adding "near Eiffel Tower" could help, but Giverny is already a specific location, so maybe it's better to keep it simple. I don't want to overcomplicate the query.

Once I run the search, the tool will return the results. I should look for hotels that are in Giver

2025-07-28 14:41:49,280 - __main__ - INFO - 🤖 AI Response: Based on the search results, here are hotels in Giverny with free breakfast: [list from search results]
2025-07-28 14:41:49,281 - __main__ - INFO - ✅ Query completed successfully


Final Answer: Based on the search results, here are hotels in Giverny with free breakfast: [list from search results]

> Finished chain.


## Test 2: Hotel Search in Glossop

Search for hotels in Glossop with free internet access.


In [7]:
result2 = run_hotel_query(
    eval_queries[1],  # Glossop with free internet access
    agent_executor,
    application_span
)


2025-07-28 14:41:49,288 - __main__ - INFO - 🔍 Hotel Query: I need a hotel in Glossop with free internet access




> Entering new AgentExecutor chain...
Okay, so I need to find a hotel in Glossop with free internet access. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity to find hotels based on the query. 

I should start by constructing the query. The location is Glossop, and the key amenity is free internet access. Maybe I can phrase it as "Glossop hotels with free internet". That should cover both the location and the specific amenity they're looking for.

I need to make sure I follow the format exactly. So, the first thought is to use the search tool. Then, the action is to call search_vector_database with the query. After that, I wait for the observation from the tool. Finally, I'll present the results based on that observation.

I should double-check that I'm not adding any extra text or explanations, just the required format. Also, ensure that the search query is precise to get accurate 

2025-07-28 14:42:10,956 - __main__ - INFO - 🤖 AI Response: Based on the search results, there are no hotels found for 'Glossop hotels with free internet'.
2025-07-28 14:42:10,956 - __main__ - INFO - ✅ Query completed successfully


Okay, so I need to find a hotel in Glossop with free internet access. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity to find hotels based on the query.

I should start by constructing the query. The location is Glossop, and the key amenity is free internet access. Maybe I can phrase it as "Glossop hotels with free internet". That should cover both the location and the specific amenity they're looking for.

I need to make sure I follow the format exactly. So, the first thought is to use the search tool. Then, the action is to call search_vector_database with the query. After that, I wait for the observation from the tool. Finally, I'll present the results based on that observation.

I should double-check that I'm not adding any extra text or explanations, just the required format. Also, ensure that the search query is precise to get accurate results. Maybe I can also consider if the

## Test 3: Hotel Search in Helensburgh

Search for hotels in Helensburgh with free breakfast.


In [8]:
result3 = run_hotel_query(
    eval_queries[2],  # Helensburgh with free breakfast
    agent_executor,
    application_span
)


2025-07-28 14:42:10,972 - __main__ - INFO - 🔍 Hotel Query: Show me hotels in Helensburgh with free breakfast




> Entering new AgentExecutor chain...
Okay, so I need to find hotels in Helensburgh that offer free breakfast. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity to find relevant hotels based on the query. 

I should start by crafting a query that accurately reflects what I'm looking for. The location is Helensburgh, and the key amenity is free breakfast. So, I'll phrase the query as "Helensburgh hotels with free breakfast." This should help the tool understand the exact requirements.

Next, I'll use the search_vector_database tool with this query. The tool will return the results, which I'll need to process. Once I have the results, I can present them to the user as the final answer.

I should make sure to follow the format strictly, starting with the thought, then the action, followed by the observation, and ending with the final answer. No extra information or explanations should 

2025-07-28 14:42:29,317 - __main__ - INFO - 🤖 AI Response: Based on the search results, there are no hotels found for 'Helensburgh hotels with free breakfast'.
2025-07-28 14:42:29,317 - __main__ - INFO - ✅ Query completed successfully


Okay, so I need to find hotels in Helensburgh that offer free breakfast. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity to find relevant hotels based on the query.

I should start by crafting a query that accurately reflects what I'm looking for. The location is Helensburgh, and the key amenity is free breakfast. So, I'll phrase the query as "Helensburgh hotels with free breakfast." This should help the tool understand the exact requirements.

Next, I'll use the search_vector_database tool with this query. The tool will return the results, which I'll need to process. Once I have the results, I can present them to the user as the final answer.

I should make sure to follow the format strictly, starting with the thought, then the action, followed by the observation, and ending with the final answer. No extra information or explanations should be added.

Final Answer: Based on the sea

## Arize Phoenix Evaluation

This section demonstrates how to evaluate the hotel support agent using Arize Phoenix observability platform. The evaluation includes:

- **Relevance Scoring**: Using Phoenix RelevanceEvaluator to score how relevant responses are to queries
- **QA Scoring**: Using Phoenix QAEvaluator to score answer quality
- **Hallucination Detection**: Using Phoenix HallucinationEvaluator to detect fabricated information  
- **Toxicity Detection**: Using Phoenix ToxicityEvaluator to detect harmful content
- **Phoenix UI**: Real-time observability dashboard at `http://localhost:6006/`

We'll run hotel search queries and evaluate the responses for quality and safety.


In [ ]:
# Import Phoenix evaluation components and queries system
try:
    import phoenix as px
    from phoenix.evals import (
        HALLUCINATION_PROMPT_RAILS_MAP,
        HALLUCINATION_PROMPT_TEMPLATE,
        QA_PROMPT_RAILS_MAP,
        QA_PROMPT_TEMPLATE,
        RAG_RELEVANCY_PROMPT_RAILS_MAP,
        RAG_RELEVANCY_PROMPT_TEMPLATE,
        TOXICITY_PROMPT_RAILS_MAP,
        TOXICITY_PROMPT_TEMPLATE,
        OpenAIModel,
        llm_classify,
    )
    import pandas as pd
    
    # Import queries system
    sys.path.append(os.path.join(os.getcwd(), "data"))
    from queries import get_evaluation_queries, get_reference_answer
    
    ARIZE_AVAILABLE = True
    logger.info("✅ Arize Phoenix evaluation components available")

except ImportError as e:
    logger.warning(f"Arize dependencies not available: {e}")
    logger.warning("Skipping evaluation section...")
    ARIZE_AVAILABLE = False

if ARIZE_AVAILABLE:
    # Start Phoenix session for observability
    try:
        px.launch_app(port=6006)
        logger.info("🚀 Phoenix UI available at http://localhost:6006/")
    except Exception as e:
        logger.warning(f"Could not start Phoenix UI: {e}")

    # Get evaluation queries - use first working query for demo
    all_eval_queries = get_evaluation_queries()
    hotel_demo_queries = [all_eval_queries[0]]  # Use first query: Giverny with free breakfast
    
    # Run demo queries and collect responses for evaluation
    hotel_demo_results = []
    
    for i, query in enumerate(hotel_demo_queries, 1):
        try:
            logger.info(f"🔍 Running evaluation query {i}: {query}")
            
            # Run the agent
            response = agent_executor.invoke({"input": query})
            output = response.get("output", "No response generated")
    
            hotel_demo_results.append({
                "query": query,
                "response": output,
                "query_type": f"hotel_demo_{i}",
                "success": True
            })
            
            logger.info(f"✅ Query {i} completed successfully")
    
        except Exception as e:
            logger.exception(f"❌ Query {i} failed: {e}")
            hotel_demo_results.append({
                "query": query,
                "response": f"Error: {e!s}",
                "query_type": f"hotel_demo_{i}",
                "success": False
            })
    
    # Convert to DataFrame for evaluation
    hotel_results_df = pd.DataFrame(hotel_demo_results)
    logger.info(f"📊 Collected {len(hotel_results_df)} responses for evaluation")
    
    # Display results summary
    for _, row in hotel_results_df.iterrows():
        logger.info(f"Query: {row['query']}")
        logger.info(f"Response: {row['response'][:200]}...")
        logger.info(f"Success: {row['success']}")
        logger.info("-" * 50)
    
    logger.info("💡 Visit Phoenix UI at http://localhost:6006/ to see detailed traces and evaluations")
    logger.info("💡 Use the evaluation script at evals/eval_arize.py for comprehensive evaluation")

else:
    logger.info("Arize evaluation not available - install phoenix-evals to enable evaluation")


/Users/kaustavghosh/Desktop/agent-catalog-quickstart/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-28 14:42:30,494 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
2025-07-28 14:42:30,511 - phoenix.inferences.inferences - INFO - Dataset: phoenix_inferences_720b9b00-97ee-434b-b8fc-93bfb017792e initialized
2025-07-28 14:42:35,352 - __main__ - INFO - ✅ Arize Phoenix evaluation components available
2025-07-28 14:42:35,353 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
2025-07-28 14:42:35,503 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2025-07-28 14:42:35,504 - alembic.runtime.migration - INFO - Will assume transactional DDL.
2025-07-28 14:42:35,533 - alembic.runtime.migration - INFO - 

❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


2025-07-28 14:42:35,683 - alembic.runtime.migration - INFO - Running upgrade cf03bd6bae1d -> 10460e46d750, datasets
2025-07-28 14:42:35,689 - alembic.runtime.migration - INFO - Running upgrade 10460e46d750 -> 3be8647b87d8, add token columns to spans table
2025-07-28 14:42:35,692 - alembic.runtime.migration - INFO - Running upgrade 3be8647b87d8 -> cd164e83824f, users and tokens
2025-07-28 14:42:35,697 - alembic.runtime.migration - INFO - Running upgrade cd164e83824f -> 4ded9e43755f, create project_session table
2025-07-28 14:42:35,706 - alembic.runtime.migration - INFO - Running upgrade 4ded9e43755f -> bc8fea3c2bc8, Add prompt tables
2025-07-28 14:42:35,711 - alembic.runtime.migration - INFO - Running upgrade bc8fea3c2bc8 -> 2f9d1a65945f, Annotation config migrations
/Users/kaustavghosh/.pyenv/versions/3.12.8/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/Users/kaustavghosh/.pyen

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


> Entering new AgentExecutor chain...
Okay, so I need to find hotels in Giverny with free breakfast. First, I should figure out the best way to search for this. I remember that the tool available is search_vector_database, which uses semantic similarity. That means I should phrase my query in a way that matches what people might search for when looking for hotels.

I think starting with "Giverny hotels with free breakfast" makes sense. It's straightforward and covers both the location and the specific amenity they're looking for. I should make sure to include all the important keywords so the tool can find the most relevant results.

I wonder if there are any other ways to phrase it, but I think this is clear enough. Maybe adding "near Eiffel Tower" could help, but Giverny is already a specific location, so maybe it's better to

2025-07-28 14:42:55,026 - __main__ - INFO - ✅ Query 1 completed successfully
2025-07-28 14:42:55,027 - __main__ - INFO - 📊 Collected 1 responses for evaluation
2025-07-28 14:42:55,028 - __main__ - INFO - Query: Find hotels in Giverny with free breakfast
2025-07-28 14:42:55,029 - __main__ - INFO - Response: Based on the search results, there are no hotels found for 'Giverny hotels with free breakfast'....
2025-07-28 14:42:55,029 - __main__ - INFO - Success: True
2025-07-28 14:42:55,029 - __main__ - INFO - --------------------------------------------------
2025-07-28 14:42:55,029 - __main__ - INFO - 💡 Visit Phoenix UI at http://localhost:6006/ to see detailed traces and evaluations
2025-07-28 14:42:55,030 - __main__ - INFO - 💡 Use the evaluation script at evals/eval_arize.py for comprehensive evaluation


Okay, so I need to find hotels in Giverny with free breakfast. I'll start by using the search_vector_database tool with the query "Giverny hotels with free breakfast". If the tool returns results, I'll list them. If not, I'll let the user know there are no available hotels. 

Final Answer: Based on the search results, there are no hotels found for 'Giverny hotels with free breakfast'.

> Finished chain.


llm_classify |          | 0/1 (0.0%) | ⏳ 00:04<? | ?it/s


In [10]:
if ARIZE_AVAILABLE and len(hotel_demo_results) > 0:
    logger.info("🔍 Running comprehensive Phoenix evaluations...")
    
    # Setup evaluator LLM (using OpenAI for consistency)
    evaluator_llm = OpenAIModel(model="gpt-4o", temperature=0.0)
    
    # Prepare evaluation data with proper column names for Phoenix evaluators
    hotel_eval_data = []
    for _, row in hotel_results_df.iterrows():
        query = row["query"]
        reference = get_reference_answer(query)
        hotel_eval_data.append({
            "input": query,
            "output": row["response"],
            "reference": reference,
            "text": row["response"]  # For toxicity evaluation
        })
    
    hotel_eval_df = pd.DataFrame(hotel_eval_data)
    
    try:
        # 1. Relevance Evaluation
        logger.info("🔍 Running Relevance Evaluation...")
        hotel_relevance_results = llm_classify(
            data=hotel_eval_df[["input", "reference"]],  # Fixed: use 'data' instead of 'dataframe'
            model=evaluator_llm,
            template=RAG_RELEVANCY_PROMPT_TEMPLATE,
            rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
            provide_explanation=True  # Added for explanations
        )
        
        logger.info("✅ Relevance Evaluation Results:")
        # Fixed: Handle DataFrame results correctly
        if hasattr(hotel_relevance_results, "columns"):
            relevance_labels = hotel_relevance_results["label"].tolist() if "label" in hotel_relevance_results.columns else ["unknown"] * len(hotel_eval_data)
            relevance_explanations = hotel_relevance_results["explanation"].tolist() if "explanation" in hotel_relevance_results.columns else ["No explanation"] * len(hotel_eval_data)
            
            for i, label in enumerate(relevance_labels):
                query = hotel_eval_data[i]["input"]
                logger.info(f"   Query: {query}")
                logger.info(f"   Relevance: {label}")
                logger.info("   " + "-"*30)
        
        # 2. QA Evaluation
        logger.info("🔍 Running QA Evaluation...")
        hotel_qa_results = llm_classify(
            data=hotel_eval_df[["input", "output", "reference"]],  # Fixed: use 'data'
            model=evaluator_llm,
            template=QA_PROMPT_TEMPLATE,
            rails=list(QA_PROMPT_RAILS_MAP.values()),
            provide_explanation=True
        )
        
        logger.info("✅ QA Evaluation Results:")
        if hasattr(hotel_qa_results, "columns"):
            qa_labels = hotel_qa_results["label"].tolist() if "label" in hotel_qa_results.columns else ["unknown"] * len(hotel_eval_data)
            qa_explanations = hotel_qa_results["explanation"].tolist() if "explanation" in hotel_qa_results.columns else ["No explanation"] * len(hotel_eval_data)
            
            for i, label in enumerate(qa_labels):
                query = hotel_eval_data[i]["input"]
                logger.info(f"   Query: {query}")
                logger.info(f"   QA Score: {label}")
                logger.info("   " + "-"*30)
        
        # 3. Hallucination Evaluation
        logger.info("🔍 Running Hallucination Evaluation...")
        hotel_hallucination_results = llm_classify(
            data=hotel_eval_df[["input", "reference", "output"]],  # Fixed: use 'data'
            model=evaluator_llm,
            template=HALLUCINATION_PROMPT_TEMPLATE,
            rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values()),
            provide_explanation=True
        )
        
        logger.info("✅ Hallucination Evaluation Results:")
        if hasattr(hotel_hallucination_results, "columns"):
            hallucination_labels = hotel_hallucination_results["label"].tolist() if "label" in hotel_hallucination_results.columns else ["unknown"] * len(hotel_eval_data)
            hallucination_explanations = hotel_hallucination_results["explanation"].tolist() if "explanation" in hotel_hallucination_results.columns else ["No explanation"] * len(hotel_eval_data)
            
            for i, label in enumerate(hallucination_labels):
                query = hotel_eval_data[i]["input"]
                logger.info(f"   Query: {query}")
                logger.info(f"   Hallucination: {label}")
                logger.info("   " + "-"*30)
        
        # 4. Toxicity Evaluation
        logger.info("🔍 Running Toxicity Evaluation...")
        hotel_toxicity_results = llm_classify(
            data=hotel_eval_df[["text"]],  # Fixed: use 'data'
            model=evaluator_llm,
            template=TOXICITY_PROMPT_TEMPLATE,
            rails=list(TOXICITY_PROMPT_RAILS_MAP.values()),
            provide_explanation=True
        )
        
        logger.info("✅ Toxicity Evaluation Results:")
        if hasattr(hotel_toxicity_results, "columns"):
            toxicity_labels = hotel_toxicity_results["label"].tolist() if "label" in hotel_toxicity_results.columns else ["unknown"] * len(hotel_eval_data)
            toxicity_explanations = hotel_toxicity_results["explanation"].tolist() if "explanation" in hotel_toxicity_results.columns else ["No explanation"] * len(hotel_eval_data)
            
            for i, label in enumerate(toxicity_labels):
                query = hotel_eval_data[i]["input"]
                logger.info(f"   Query: {query}")
                logger.info(f"   Toxicity: {label}")
                logger.info("   " + "-"*30)
        
        # Summary of all evaluations
        logger.info("📊 EVALUATION SUMMARY")
        logger.info("=" * 50)
        
        for i, query in enumerate([item["input"] for item in hotel_eval_data]):
            logger.info(f"Query {i+1}: {query}")
            logger.info(f"  Relevance: {relevance_labels[i] if 'relevance_labels' in locals() else 'N/A'}")
            logger.info(f"  QA Score: {qa_labels[i] if 'qa_labels' in locals() else 'N/A'}")
            logger.info(f"  Hallucination: {hallucination_labels[i] if 'hallucination_labels' in locals() else 'N/A'}")
            logger.info(f"  Toxicity: {toxicity_labels[i] if 'toxicity_labels' in locals() else 'N/A'}")
            logger.info("  " + "-"*40)
        
        logger.info("✅ All Phoenix evaluations completed successfully!")
        
    except Exception as e:
        logger.exception(f"❌ Phoenix evaluation failed: {e}")
        logger.info("💡 This might be due to API rate limits or model availability")
        
else:
    if not ARIZE_AVAILABLE:
        logger.info("❌ Phoenix evaluations skipped - Arize dependencies not available")
    else:
        logger.info("❌ Phoenix evaluations skipped - No demo results to evaluate")

2025-07-28 14:42:55,051 - __main__ - INFO - 🔍 Running comprehensive Phoenix evaluations...
2025-07-28 14:42:55,066 - __main__ - INFO - 🔍 Running Relevance Evaluation...
2025-07-28 14:42:55,082 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:02<00:00 |  2.90s/it
2025-07-28 14:42:57,980 - __main__ - INFO - ✅ Relevance Evaluation Results:
2025-07-28 14:42:57,981 - __main__ - INFO -    Query: Find hotels in Giverny with free breakfast
2025-07-28 14:42:57,981 - __main__ - INFO -    Relevance: relevant
2025-07-28 14:42:57,981 - __main__ - INFO -    ------------------------------
2025-07-28 14:42:57,981 - __main__ - INFO - 🔍 Running QA Evaluation...
2025-07-28 14:42:57,994 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event

Retries exhausted after 1 attempts: Missing template variable: 'input'


## Summary

This notebook demonstrates a complete hotel support agent implementation using:

1. **Agent Catalog Integration**: Using agentc to find tools and prompts
2. **LangChain Framework**: ReAct agent pattern with Agent Catalog callbacks
3. **Couchbase Vector Store**: Storing and searching hotel data from travel-sample bucket
4. **Single Tool Architecture**: Focused on `search_vector_database` for hotel search
5. **Comprehensive Evaluation**: Phoenix-based evaluation with multiple metrics

The agent can handle various hotel-related queries including:
- Hotel search by location and amenities
- Finding hotels with specific features (parking, internet, spa, etc.)
- Luxury hotel searches with premium amenities
- Location-based hotel recommendations

## Phoenix Evaluation Metrics

The notebook demonstrates all four key Phoenix evaluation types:

1. **Relevance Evaluation**: Measures how relevant responses are to user queries
2. **QA Evaluation**: Assesses the quality and accuracy of answers
3. **Hallucination Detection**: Identifies fabricated or incorrect information
4. **Toxicity Detection**: Screens for harmful or inappropriate content

Each evaluation provides:
- Binary or categorical labels (e.g., "relevant"/"irrelevant", "correct"/"incorrect")
- Detailed explanations of the evaluation reasoning
- Confidence scores for the assessments

## Key Features

This hotel support agent implementation:
- **Uses travel-sample bucket**: Leverages existing Couchbase sample data
- **Capella AI integration**: Supports both embeddings and LLM through Capella AI
- **Fallback to OpenAI**: Graceful fallback when Capella AI is unavailable
- **Single focused tool**: Simplified architecture with one search tool
- **Comprehensive evaluation**: Full Phoenix evaluation pipeline
- **Agent Catalog callbacks**: Integrated observability and tracing

## Data Source

The agent uses hotel data from the `travel-sample.inventory.hotel` collection, which contains:
- Real hotel information with names, locations, and amenities
- Structured data with address, city, country, and facilities
- Rich text descriptions suitable for vector embedding
- Comprehensive coverage of global hotel properties

For production use, consider:
- Setting up proper monitoring with Arize Phoenix
- Implementing comprehensive evaluation pipelines
- Adding error handling and retry logic
- Scaling the vector store for larger datasets
- Adding more sophisticated query understanding

## Usage Instructions

To run this notebook:
1. Set up the required environment variables (Couchbase connection, API keys)
2. Install dependencies: `pip install -r requirements.txt`
3. Ensure travel-sample bucket is available in your Couchbase cluster
4. Publish your agent catalog: `agentc index . && agentc publish`
5. Run the notebook cells sequentially

The agent will automatically load hotel data from travel-sample and create embeddings for semantic search capabilities.
